In [ ]:
import numpy as np
from geopy.geocoders import Nominatim
import osmnx as ox
import networkx as nx
from scipy.optimize import linear_sum_assignment
import folium
import pandas as pd

### Olvasd be a címeket.

Az első tíz sorban a taxik, utána az utasok helyzete van megadva 

In [ ]:
def get_addresses(input_path: str) -> tuple[list[str], list[str]]:
    #########
    # TO DO #
    #########
    return taxis, clients


In [ ]:
taxis, clients = get_addresses("addresses.txt")
print(taxis)
print(clients)

### A geopy csomag használatával add meg a taxik és ügyfelek helyzetének koordinátáit!

In [ ]:
def get_coords(address: str) -> tuple[float, float]:
    #########
    # TO DO #
    #########
    return latitude, longitude

In [ ]:
taxi_co = [get_coords(a) for a in taxis]
client_co = [get_coords(a) for a in clients]

In [ ]:
print(taxi_co)
print(client_co)

### Vizualizáció - ezen nem kell módosítanod, csak futtasd le, hogy térképen lásd a koordinátákat

In [ ]:
def show_map(taxi_co: list, taxis: list, client_co: list, clients: list, routes: list):
    if taxi_co is not None:
        ref_point = taxi_co[0]
    else:
        ref_point = client_co[0]
    m = folium.Map(location = [ref_point[0], ref_point[1]], zoom_start = 13)
    if taxi_co is not None:
        for i in range(len(taxi_co)):
            folium.Marker(location=[taxi_co[i][0], taxi_co[i][1]], popup=taxis[i], icon=folium.Icon(color='green')).add_to(m)
    if client_co is not None:
        for i in range(len(client_co)):
    
            folium.Marker(location=[client_co[i][0], client_co[i][1]], popup=clients[i], icon=folium.Icon(color='blue')).add_to(m)
    if routes is not None:
        for route in routes:
            route_coords = [(G.nodes[node]['y'], G.nodes[node]['x']) for node in route]
            #route_coords = [(parlament.latitude, parlament.longitude)] + route_coords + [(oktogon.latitude, oktogon.longitude)]
            folium.PolyLine(route_coords, color="green", weight=5, opacity=0.8).add_to(m)
    return m

In [ ]:
m = show_map(taxi_co, taxis, client_co, clients, None)
m

# Számold ki a távolságokat (legrövidebb út) a lokációk között

Használd az osmnx csomagot!

In [ ]:
def load_map() -> nx.classes.multidigraph.MultiDiGraph:
    #########
    # TO DO #
    #########
    return G

def shortest_paths(G, taxi_co, client_co) -> np.ndarray:
    distance_matrix = np.zeros((10,10))
    #########
    # TO DO #
    #########
    return distance_matrix

In [ ]:
G = load_map()
distance_matrix = shortest_paths(G,taxi_co, client_co)

In [ ]:
# Look at the distance matrix:
# pd.DataFrame(distance_matrix.round(3), index = taxis, columns = clients)

# Optimális párosítás

Használd a scipy csomag megfelelő függvényét!

In [ ]:
def optimal_assignment(distance_matrix, taxi_co, client_co) -> tuple[float, list[tuple]]:
    #########
    # TO DO #
    #########
    return total_distance, optimal_assignments_co


A függvény outputja így nézzen ki:
 - total_distance: float, az összes távolság az optimálisan a taxik és a hozzájuk párosított utasok között
 - optimal_assignments_co: egy lista, amely a taxi-ügyfél párosítások koordinátáit tartalmazza
```
total_distance, [
    (taxi_coords, client_coords),
    (taxi_coords, client_coords),
    (taxi_coords, client_coords),
    ...   
]

Például:
28.623, [
    ((47.5, 19.2), (46.4, 20.4)),
    ((47.6, 18.1), (48.3, 19.8)),
    ((47.9, 20.2), (49.4, 20.6)),  
]
```


In [ ]:
total_distance, optimal_assignments_co = optimal_assignment(distance_matrix)
print(total_distance)

### Vizualizáció - ezen nem kell dolgoznod, csak futtasd le, hogy lásd térképen is az optimális párosítást és útvonalakat

In [ ]:
optimal_routes = []
for taxi, client in optimal_assignments_co:
    orig_node = ox.distance.nearest_nodes(G, taxi[1], taxi[0])
    dest_node = ox.distance.nearest_nodes(G, client[1], client[0])
    route = nx.shortest_path(G, orig_node, dest_node, weight='length')
    optimal_routes.append(route)

m = show_map(taxi_co, taxis, client_co, clients, optimal_routes)
m